# Anomalie globale de température (HadCRUT 1850-2022)

**Auteur : Frédéric FERRY (ENM) - septembre 2023**

Données HadCRUT5 : https://www.metoffice.gov.uk/hadobs/hadcrut5/data/current/download.html

- https://www.metoffice.gov.uk/hadobs/hadcrut5/data/current/analysis/HadCRUT.5.0.1.0.analysis.anomalies.ensemble_mean.nc
- https://www.metoffice.gov.uk/hadobs/hadcrut5/data/current/analysis/diagnostics/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.nc

Graphiques "en bandes" d'après #ShowYourStripes d'Ed Hawkins (University of Reading) : https://showyourstripes.info

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib import animation

import pandas as pd
import xarray as xr

from cartopy import config
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes

from tqdm import tqdm

In [ ]:
t    = xr.open_dataset("./data/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.nc")
print(t)

In [ ]:
# Reference period for the center of the color scale
FIRST_REFERENCE = 1971
LAST_REFERENCE = 2000
LIM = 0.7 # degrees

FIRST = '1850'
LAST = '2022'  # inclusive

# Reference period for the center of the color scale
FIRST_REFERENCE = '1971'
LAST_REFERENCE = '2000'

t_anom = t.sel(time=slice(FIRST, LAST))
t_ref = t_anom.sel(time=slice(FIRST_REFERENCE, LAST_REFERENCE)).mean('time')

anomaly=t_anom['tas_mean']
reference=t_ref['tas_mean']

#print(reference)

# the 8 more saturated colors from the 9 blues / 9 reds
cmap = ListedColormap([
    '#08306b', '#08519c', '#2171b5', '#4292c6',
    '#6baed6', '#9ecae1', '#c6dbef', '#deebf7',
    '#fee0d2', '#fcbba1', '#fc9272', '#fb6a4a',
    '#ef3b2c', '#cb181d', '#a50f15', '#67000d',
])

fig = plt.figure(figsize=(15, 5))
fig.suptitle('HadCRUT 5 Global Temperature anomaly from '+FIRST+' to '+LAST+' (relative to '+FIRST_REFERENCE+'-'+LAST_REFERENCE+')'
             ,fontsize=12)

#ax = fig.add_axes([0, 0, 1, 1])
ax=fig.add_subplot(111)

# create a collection with a rectangle for each year
col = PatchCollection([Rectangle((y, 0), 1, 1) for y in range(int(FIRST), int(LAST) + 1) ])

# set data, colormap and color limits
col.set_array(anomaly)
col.set_cmap(cmap)
col.set_clim(reference - LIM, reference + LIM)
ax.add_collection(col)
ax.set_ylim(0, 1)
ax.set_xlim(int(FIRST), int(LAST) + 1)

#ax.set_axis_off()
ax.get_yaxis().set_visible(False)
plt.colorbar(col, orientation="horizontal", aspect=65, shrink=0.5, pad=0.1)

fig.savefig('./figs/HadCRUT5_Global_Tanom_warming-stripes.png')

In [ ]:
t    = xr.open_dataset("./data/HadCRUT.5.0.1.0.analysis.anomalies.ensemble_mean.nc")
print(t)
tyear=t.sel(time=slice('1850','2022')).groupby('time.year').mean('time', skipna=True)

years=tyear.year.values
lat=tyear.latitude.values

lon=tyear.longitude.values
print(years)

data=tyear['tas_mean']
print(data.shape)

In [ ]:
def plot_background(ax):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    return ax

In [ ]:
#plt.style.use('dark_background')

proj=ccrs.EqualEarth()

levels=np.arange(-1,1.1, 0.1)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('HadCRUT 5.0 Temperature anomaly (°C relative to 1961-1990)', fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(17, 10),
       axes_pad=0.2,
       cbar_location='bottom',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

start,stop = 0,len(years)-1

for i, ax in enumerate(axgr[:stop-start]):
    plot_background(ax)
    ax.set_title(str(years[i]), fontsize=10)
    p1 = ax.contourf(lon, lat, data[i,:,:], levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
    #p2 = ax.contour(lon, lat, data[i,:,:], colors='black', levels=levels, linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname='./figs/HadCRUT5_Tanom'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
iyear=int(input("Entrer l'année souhaitée : "))
idx = list(years).index(iyear)

proj=ccrs.EqualEarth()

levels=np.arange(-2,2.1, 0.1)

fig = plt.figure(figsize=(15,10))
fig.suptitle('HadCRUT 5.0 Temperature anomaly (°C relative to 1961-1990)', fontsize=16)
ax = fig.add_subplot(1, 1, 1, projection=proj)
plot_background(ax)
ax.set_title(str(iyear), fontsize=14)
p1 = ax.contourf(lon, lat, data[idx,:,:], levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
#p2 = ax.contour(lon, lat, data[idx,:,:], colors='black', levels=levels, linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(p1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
cb.set_label('°C', size='small')   
plt.show()

figname='./figs/HadCRUT5_Tanom_'+str(iyear)
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
for i in tqdm(range(len(years))): 
    fig = plt.figure(figsize=(15,10))
    fig.suptitle('HadCRUT 5.0 Temperature anomaly (°C relative to 1961-1990)', fontsize=16)
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    plot_background(ax)
    ax.set_global()
    ax.set_title(str(years[i]), fontsize=14)
    p1 = ax.contourf(lon, lat, data[i,:,:], levels=levels, transform=ccrs.PlateCarree(),
                     cmap='coolwarm', extend='both')
    #p2 = ax.contour(lon, lat, data[i,:,:], colors='black', levels=levels, linewidths=0.2, transform=ccrs.PlateCarree())
    cb = fig.colorbar(p1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
    cb.set_label('°C', size='small')   
    figname='./anim/HadCRUT5_Tanom_'+str(years[i])
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
gif_filepath = './anim/HadCRUT5_Tanom.gif'
make_animation(gif_filepath)